In [34]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

## Correct pre-processing

In [172]:
folder_path = '/home/melissa.lopez/GlitchBank/LargeAnalysis/O3aGB/FinalData/'
detector = 'L1'

In [173]:
# Define some lists
blip_files, fast_scattering_files, injection_files = list(), list(), list()
koyfish_files, lowfreq_files, tomte_files = list(), list(), list()
whistle_files = list()

# Iterate over all files in the given folder
for filename in np.sort(os.listdir(folder_path)):
    # Check if the file is a CSV file
    if filename.endswith(".csv"):
        #Check what type of data the file contains and put in in the corresponding list.
        if filename.startswith("Blip_" + detector):
            blip_files.append((os.path.join(folder_path, filename)))                     
        elif filename.startswith("Injections_" + detector):
            injection_files.append((os.path.join(folder_path, filename)))       
        elif filename.startswith("Koi_Fish_" + detector):
            koyfish_files.append((os.path.join(folder_path, filename)))
        elif filename.startswith("Low_Frequency_Burst_" + detector):
            lowfreq_files.append((os.path.join(folder_path, filename)))
        elif filename.startswith("Tomte_" + detector):
            tomte_files.append((os.path.join(folder_path, filename)))
        elif filename.startswith("Whistle_" + detector):
            whistle_files.append((os.path.join(folder_path, filename))) 
        elif filename.startswith("Fast_Scattering_" + detector) and (detector != "V1"):
            fast_scattering_files.append((os.path.join(folder_path, filename)))

In [12]:
path = injection_files[0]
df = pd.read_csv(path)

In [113]:
df1_grouped = df.groupby(['Cluster ID', 'Cluster time'])

In [191]:
def averageFeature(cols, group_name, tmp, classes):
    feature = np.zeros((1, len(cols)))

    for c in range(len(cols)-1):
        feature[0, c] = (tmp[cols[c]] * weights).sum() / weights.sum()
    feature[0, len(cols) - 1] = int(classes)
    feature = pd.DataFrame(feature, columns=cols)

    return feature

In [190]:
drop_cols = ['Unnamed: 0', 'ifo', 'Trigger ID', 'Trigger time', 'Cluster ID', 'Cluster time', 'Template ID']
k = 0
cluster_id, cluster_time = list(), list()
for group_name, df_group in df1_grouped:
    print(group_name)
    cluster_id.append(group_name[0])
    cluster_time.append(group_name[1])
    tmp = df_group.sort_values(by='Trigger time')
    time = tmp['Trigger time']
    tmp = tmp.drop(drop_cols, axis=1)
    weights = tmp['SNR'] / np.max(tmp['SNR'])
    print(len(tmp['SNR']), np.mean(tmp['SNR']))
    cols = list(tmp.columns)
    cols.extend(['Class'])
    classes = 0
    feature = averageFeature(cols, group_name, tmp, classes)
    
    if k == 0:
        features = feature
    else:
        features = pd.concat([features, feature])
    k = k + 1
    display(features)
    if k == 2:
        break

IDs = pd.DataFrame({'Cluster ID':cluster_id, 'Cluster time':cluster_time})

(2, 1238166114.4367027)
64 7.5837232859374994


,SNR,Chisq,Mass_1,Mass_2,Spin1z,Spin2z,Class
0,7.850731,2.769055,174.809988,37.57883,-0.28955,0.06697,0.0


(2, 1244451019.3194437)
117 16.27856853931624


,SNR,Chisq,Mass_1,Mass_2,Spin1z,Spin2z,Class
0,7.850731,2.769055,174.809988,37.578830,-0.289550,0.066970,0.0
0,16.747768,3.476546,62.979454,11.773144,-0.483233,-0.055422,0.0


In [192]:
def classTag(file_name):
    if 'Injections' in file_name:
        t = 0
    if 'Blip' in file_name:
        t = 1
    if 'Koi_Fish' in file_name:
        t = 2
    if 'Low_Frequency_Burst' in file_name:
        t = 3
    if 'Tomte' in file_name:
        t = 4
    if 'Whistle' in file_name:
        t = 5
    if 'Fast_Scattering' in file_name:
        t = 6
    return t

In [193]:
file_list = np.sort(os.listdir(folder_path))
for file_name in file_list:
    if detector in file_name:
        t = classTag(file_name)
        print(file_name, t)

Blip_L1_O3a.csv 1
Blip_L1_O3b.csv 1
Fast_Scattering_L1_O3a.csv 6
Fast_Scattering_L1_O3b.csv 6
Injections_L1_O3a.csv 0
Injections_L1_O3b.csv 0
Koi_Fish_L1_O3a.csv 2
Koi_Fish_L1_O3b.csv 2
Low_Frequency_Burst_L1_O3a.csv 3
Low_Frequency_Burst_L1_O3b.csv 3
Tomte_L1_O3a.csv 4
Tomte_L1_O3b.csv 4
Whistle_L1_O3a.csv 5
Whistle_L1_O3b.csv 5


In [174]:
blip_files

['/home/melissa.lopez/GlitchBank/LargeAnalysis/O3aGB/FinalData/Blip_L1_O3a.csv',
 '/home/melissa.lopez/GlitchBank/LargeAnalysis/O3aGB/FinalData/Blip_L1_O3b.csv']

In [ ]:
injection_triggers = read_triggers(injection_files, 0, True)
blip_triggers = read_triggers(blip_files, 1, True)
koyfish_triggers = read_triggers(koyfish_files, 2, True)
lowfreq_triggers = read_triggers(lowfreq_files, 3, True)
tomte_triggers = read_triggers(tomte_files, 4, True)
whistle_triggers = read_triggers(whistle_files, 5, True)
if detector != 'V1':
    fast_scattering_triggers = read_triggers(fast_scattering_files, 6, True)



## Correct boostrap

In [150]:
!ls ../../LargeAnalysis/O3aGB/AvDataFrames/

Av_Blip_L1_O3a.csv		  Av_Koi_Fish_L1_O3b.csv
Av_Blip_L1_O3a_ID.csv		  Av_Koi_Fish_L1_O3b_ID.csv
Av_Blip_L1_O3b.csv		  Av_Low_Frequency_Burst_L1_O3a.csv
Av_Blip_L1_O3b_ID.csv		  Av_Low_Frequency_Burst_L1_O3a_ID.csv
Av_Fast_Scattering_L1_O3a.csv	  Av_Low_Frequency_Burst_L1_O3b.csv
Av_Fast_Scattering_L1_O3a_ID.csv  Av_Low_Frequency_Burst_L1_O3b_ID.csv
Av_Fast_Scattering_L1_O3b.csv	  Av_Tomte_L1_O3a.csv
Av_Fast_Scattering_L1_O3b_ID.csv  Av_Tomte_L1_O3a_ID.csv
Av_Injections_L1_O3a.csv	  Av_Tomte_L1_O3b.csv
Av_Injections_L1_O3a_ID.csv	  Av_Tomte_L1_O3b_ID.csv
Av_Injections_L1_O3b.csv	  Av_Whistle_L1_O3a.csv
Av_Injections_L1_O3b_ID.csv	  Av_Whistle_L1_O3a_ID.csv
Av_Koi_Fish_L1_O3a.csv		  Av_Whistle_L1_O3b.csv
Av_Koi_Fish_L1_O3a_ID.csv	  Av_Whistle_L1_O3b_ID.csv


In [157]:
path_av = '../../LargeAnalysis/O3aGB/AvDataFrames/'
ifo = 'L1'

for file in np.sort(os.listdir(path_av)):
    if (ifo in file) and ('ID' not in file) and ('O3a' in file):
        print(file)
        

Av_Blip_L1_O3a.csv
Av_Fast_Scattering_L1_O3a.csv
Av_Injections_L1_O3a.csv
Av_Koi_Fish_L1_O3a.csv
Av_Low_Frequency_Burst_L1_O3a.csv
Av_Tomte_L1_O3a.csv
Av_Whistle_L1_O3a.csv


In [158]:
!ls ../../LargeAnalysis/O3aGB/AvDataFrames/

Av_Blip_L1_O3a.csv		  Av_Koi_Fish_L1_O3b.csv
Av_Blip_L1_O3a_ID.csv		  Av_Koi_Fish_L1_O3b_ID.csv
Av_Blip_L1_O3b.csv		  Av_Low_Frequency_Burst_L1_O3a.csv
Av_Blip_L1_O3b_ID.csv		  Av_Low_Frequency_Burst_L1_O3a_ID.csv
Av_Fast_Scattering_L1_O3a.csv	  Av_Low_Frequency_Burst_L1_O3b.csv
Av_Fast_Scattering_L1_O3a_ID.csv  Av_Low_Frequency_Burst_L1_O3b_ID.csv
Av_Fast_Scattering_L1_O3b.csv	  Av_Tomte_L1_O3a.csv
Av_Fast_Scattering_L1_O3b_ID.csv  Av_Tomte_L1_O3a_ID.csv
Av_Injections_L1_O3a.csv	  Av_Tomte_L1_O3b.csv
Av_Injections_L1_O3a_ID.csv	  Av_Tomte_L1_O3b_ID.csv
Av_Injections_L1_O3b.csv	  Av_Whistle_L1_O3a.csv
Av_Injections_L1_O3b_ID.csv	  Av_Whistle_L1_O3a_ID.csv
Av_Koi_Fish_L1_O3a.csv		  Av_Whistle_L1_O3b.csv
Av_Koi_Fish_L1_O3a_ID.csv	  Av_Whistle_L1_O3b_ID.csv


In [171]:
np.load('../../LargeAnalysis/O3aGB/AvDataFrames/dataset_all_L1_bootstrap_O3a.npy')[:,-1]

array([-0.58485679,  0.2800241 ,  0.89622325, ..., -0.42957601,
        0.25144339,  0.04702031])

In [ ]:
injections = np.load("../datasets/injection_triggers_" + detector + ".npy").tolist()
blips = np.load("../datasets/blip_triggers_" + detector + ".npy").tolist()
koyfishes = np.load("../datasets/koyfish_triggers_" + detector + ".npy").tolist()
lowfreqs = np.load("../datasets/lowfreq_triggers_" + detector + ".npy").tolist()
tomtes = np.load("../datasets/tomte_triggers_" + detector + ".npy").tolist()
whistles = np.load("../datasets/whistle_triggers_" + detector + ".npy").tolist()

In [194]:
np.ones(2)

array([1., 1.])